In [1]:
!pip install -qq --upgrade transformers accelerate diffusers
!rm -r sample_data

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 13.8 MB/s eta 0:00:00


In [2]:
# Set the details for your model here:
import torch

from diffusers import StableDiffusionXLPipeline, StableDiffusionXLImg2ImgPipeline, AutoencoderKL, KDPM2AncestralDiscreteScheduler

vae = AutoencoderKL.from_pretrained("madebyollin/sdxl-vae-fp16-fix", torch_dtype=torch.float16)
base2 = StableDiffusionXLImg2ImgPipeline.from_pretrained("stabilityai/stable-diffusion-xl-base-1.0",
    vae=vae,
    torch_dtype=torch.float16,
    variant="fp16",
    use_safetensors=True,
)

_ = base2.to("cuda")

torch.cuda.empty_cache()

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

config.json:   0%|          | 0.00/631 [00:00<?, ?B/s]

diffusion_pytorch_model.safetensors:   0%|          | 0.00/335M [00:00<?, ?B/s]

model_index.json:   0%|          | 0.00/609 [00:00<?, ?B/s]

...

tokenizer_2/vocab.json:   0%|          | 0.00/1.06M [00:00<?, ?B/s]

model.fp16.safetensors:   0%|          | 0.00/1.39G [00:00<?, ?B/s]

diffusion_pytorch_model.fp16.safetensors:   0%|          | 0.00/5.14G [00:00<?, ?B/s]

diffusion_pytorch_model.fp16.safetensors:   0%|          | 0.00/167M [00:00<?, ?B/s]

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

In [3]:
refiner = True
if refiner:
    refiner = StableDiffusionXLImg2ImgPipeline.from_pretrained("stabilityai/stable-diffusion-xl-refiner-1.0",
        text_encoder_2=base2.text_encoder_2,
        vae=vae,
        torch_dtype=torch.float16,
        use_safetensors=True,
        variant="fp16"
    )
    _ = refiner.to("cuda")

model_index.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

Fetching 9 files:   0%|          | 0/9 [00:00<?, ?it/s]

tokenizer_2/special_tokens_map.json:   0%|          | 0.00/460 [00:00<?, ?B/s]

scheduler/scheduler_config.json:   0%|          | 0.00/479 [00:00<?, ?B/s]

tokenizer_2/merges.txt:   0%|          | 0.00/525k [00:00<?, ?B/s]

...

tokenizer_2/vocab.json:   0%|          | 0.00/1.06M [00:00<?, ?B/s]

tokenizer_2/tokenizer_config.json:   0%|          | 0.00/725 [00:00<?, ?B/s]

diffusion_pytorch_model.fp16.safetensors:   0%|          | 0.00/167M [00:00<?, ?B/s]

diffusion_pytorch_model.fp16.safetensors:   0%|          | 0.00/4.52G [00:00<?, ?B/s]

Loading pipeline components...:   0%|          | 0/5 [00:00<?, ?it/s]

In [ ]:
import os
import torch
import diffusers
import random
from PIL import Image

num_inference_steps = 50

for i in range(111):
    height = width = 1024
    num_images_per_prompt = 1
    generator = torch.Generator()
    seed = i + 1234567
    generator.manual_seed(i)
    generator2 = torch.Generator(i ^ 0xffffffff)

    parameters = {
        "prompt": "Photograph of two round panettone with white glaze,nuts,dried fruit.christmas style.minimalist.clean cut,Colorful rustic background.Cutting board,wood table,amazing.Luxury kitchen.Professional food styling Photography.Intricate details even to the smallest particle,extreme detail of the enviroment,well lit, beautiful shadows,bright,photoquality,ultra realistic,award-winning",
        "negative_prompt": "Ugly,boring,disfigured,alcohol,crazy,messy,chaotic,uneven,cinnamon,loaf,loaves,mutated,deranged,huge nuts",
        "num_inference_steps": num_inference_steps,
        "num_images_per_prompt": num_images_per_prompt,
        "guidance_scale": [6, 7, 8, 9, 10, 11][i%6],
        "output_type": "latent",
        "strength": [0.9, 0.95, 0.99, 1][i%4],
        "denoising_end": 0.8,
        "image": Image.open("panettone.jpg"),
    }

    import time

    latents = base2(**parameters, generator=generator).images
    del parameters["output_type"]
    del parameters["denoising_end"]
    parameters["denoising_start"] = 0.8
    parameters["image"] = latents
    wow = refiner(**parameters, generator=generator2).images[0]
    wow.save(f"/content/drive/MyDrive/panettone{i}.png")

The following part of your input was truncated because CLIP can only handle sequences up to 77 tokens: ['- winning']
The following part of your input was truncated because CLIP can only handle sequences up to 77 tokens: ['- winning']


  0%|          | 0/35 [00:00<?, ?it/s]

In [8]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import zipfile
import os

# Set the path to the directory containing your .png files
directory = '/content'

# Create a ZipFile object to write the compressed files
with zipfile.ZipFile('/content/images.zip', 'w') as zipf:
    # Walk through the directory and its subdirectories
    for foldername, subfolders, filenames in os.walk(directory):
        for filename in filenames:
            # Check if the file is a .png file
            if filename.endswith('.png'):
                # Get the full path of the file
                file_path = os.path.join(foldername, filename)
                # Add the file to the zip archive
                zipf.write(file_path, os.path.relpath(file_path, directory))

print("All .png files in /content have been zipped to images.zip")


In [ ]:
rm -r /content/*.png